In [6]:
import pygame
import random

CAR_MOVEMENT_DELAY = 300  # Delay in milliseconds

def draw_grid_with_entities(screen, width, height, rows, cols, building_image, building_locations, robot_image, robot_location, police_image, police_location, car1_image, car1_image_locations, car2_image, car2_image_locations):
    """
    Draw a grid on the screen with building images, robot image, police image, car1 images, and car2 images at the specified locations.

    Parameters:
        screen (pygame.Surface): The surface to draw on.
        width (int): Width of the screen.
        height (int): Height of the screen.
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        building_image (pygame.Surface): The image of the building.
        building_locations (list): List of locations for the buildings [(row, col), ...].
        robot_image (pygame.Surface): The image of the robot.
        robot_location (tuple): The location of the robot (row, col).
        police_image (pygame.Surface): The image of the police.
        police_location (tuple): The location of the police (row, col).
        car1_image (pygame.Surface): The image of car1.
        car1_image_locations (list): List of locations for car1 images [(row, col), ...].
        car2_image (pygame.Surface): The image of car2.
        car2_image_locations (list): List of locations for car2 images [(row, col), ...].
    """
    cell_width = width // cols
    cell_height = height // rows

    screen.fill((0, 0, 0))  # Set background color to black

    for x in range(0, width + 1, cell_width):  # Increment the range by 1 to ensure the rightmost line is drawn
        pygame.draw.line(screen, (255, 255, 255), (x, 0), (x, height))  # Set border color to white
    for y in range(0, height + 1, cell_height):  # Increment the range by 1 to ensure the bottom line is drawn
        pygame.draw.line(screen, (255, 255, 255), (0, y), (width, y))  # Set border color to white

    # Adjust cell size to increase the size of the cell in the grid
    cell_width += 2
    cell_height += 2

    # Draw buildings
    for location in building_locations:
        building_x = location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
        building_y = location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
        building_image_resized = pygame.transform.scale(building_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness
        screen.blit(building_image_resized, (building_x, building_y))

    # Draw robot
    robot_x = robot_location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
    robot_y = robot_location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
    robot_image_resized = pygame.transform.scale(robot_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness
    screen.blit(robot_image_resized, (robot_x, robot_y))

    # Draw police
    police_x = police_location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
    police_y = police_location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
    police_image_resized = pygame.transform.scale(police_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness

    screen.blit(police_image_resized, (police_x, police_y))

    # Draw car1 images
    for location in car1_image_locations:
        car1_x = location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
        car1_y = location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
        car1_image_resized = pygame.transform.scale(car1_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness
        screen.blit(car1_image_resized, (car1_x, car1_y))

    # Draw car2 images
    for location in car2_image_locations:
        car2_x = location[1] * (cell_width - 2)  # Subtract 2 for spacing between cells
        car2_y = location[0] * (cell_height - 2)  # Subtract 2 for spacing between cells
        car2_image_resized = pygame.transform.scale(car2_image, (cell_width - 2, cell_height - 2))  # Subtract 2 for border thickness
        screen.blit(car2_image_resized, (car2_x, car2_y))

def generate_location(rows, cols, excluded_locations):
    """
    Generate a random location that does not overlap with the excluded locations.

    Parameters:
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        excluded_locations (list): List of locations to avoid [(row, col), ...].

    Returns:
        tuple: The random location (row, col).
    """
    while True:
        location = (random.randint(0, rows - 1), random.randint(0, cols - 1))
        if location not in excluded_locations and location != (0, 0):  # Exclude the (0, 0) location
            return location


def move_police(police_location, rows, cols, building_locations, robot_location, car1_image_locations, car2_image_locations, prev_direction=None):
    """
    Move the police randomly within the grid, ensuring it does not collide with buildings, robot, or other cars.

    Parameters:
        police_location (tuple): The current location of the police (row, col).
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        building_locations (list): List of locations for the buildings [(row, col), ...].
        robot_location (tuple): The location of the robot (row, col).
        car1_image_locations (list): List of locations for car1 images [(row, col), ...].
        car2_image_locations (list): List of locations for car2 images [(row, col), ...].
        prev_direction (tuple): Previous movement direction of the police (dx, dy).

    Returns:
        tuple: The new location of the police (row, col).
        tuple: The new movement direction of the police (dx, dy).
    """
    # List of possible movements (Right, Left, Down, Up)
    movements = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    # If previous direction is provided, prioritize that direction
    if prev_direction:
        movements.remove(prev_direction)
        movements.insert(0, prev_direction)

    # Randomize the order of movements
    random.shuffle(movements)

    # Check for nearby cars or robot
    nearby_entities = []
    for dx, dy in movements:
        new_row = police_location[0] + dx
        new_col = police_location[1] + dy

        if (new_row, new_col) == robot_location or \
           (new_row, new_col) in car1_image_locations or \
           (new_row, new_col) in car2_image_locations:
            nearby_entities.append((dx, dy))

    # If there are nearby entities, move away from them
    if nearby_entities:
        dx, dy = nearby_entities[0]  # Choose the first nearby entity to move away from
        dx *= -1  # Move in the opposite direction
        dy *= -1
        new_row = police_location[0] + dx
        new_col = police_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with buildings
        if (0 <= new_row < rows and 0 <= new_col < cols) and (new_row, new_col) not in building_locations:
            return (new_row, new_col), (dx, dy)

    # Try each movement until finding a valid one
    for dx, dy in movements:
        new_row = police_location[0] + dx
        new_col = police_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with buildings
        if not (0 <= new_row < rows and 0 <= new_col < cols) or (new_row, new_col) in building_locations:
            continue

        # If the new location is valid, return it along with the movement direction
        return (new_row, new_col), (dx, dy)

    # If all movements lead to obstacles or out of bounds, return the current location and no movement
    return police_location, (0, 0)


def move_car1(car1_location, rows, cols, building_locations, robot_location, police_location, car2_image_locations, prev_direction=None):
    """
    Move car1 randomly within the grid, ensuring it does not collide with buildings, robot, police, or other cars.

    Parameters:
        car1_location (tuple): The current location of car1 (row, col).
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        building_locations (list): List of locations for the buildings [(row, col), ...].
        robot_location (tuple): The location of the robot (row, col).
        police_location (tuple): The location of the police (row, col).
        car2_image_locations (list): List of locations for car2 images [(row, col), ...].
        prev_direction (tuple): Previous movement direction of car1 (dx, dy).

    Returns:
        tuple: The new location of car1 (row, col).
        tuple: The new movement direction of car1 (dx, dy).
    """
    # List of possible movements (Right, Left, Down, Up)
    movements = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    # If previous direction is provided, prioritize that direction
    if prev_direction:
        movements.remove(prev_direction)
        movements.insert(0, prev_direction)

    # Randomize the order of movements
    random.shuffle(movements)

    # Check for nearby obstacles
    nearby_entities = []
    for dx, dy in movements:
        new_row = car1_location[0] + dx
        new_col = car1_location[1] + dy

        if (new_row, new_col) == robot_location or \
           (new_row, new_col) == police_location or \
           (new_row, new_col) in car2_image_locations:
            nearby_entities.append((dx, dy))

    # If there are nearby entities, move away from them
    if nearby_entities:
        dx, dy = nearby_entities[0]  # Choose the first nearby entity to move away from
        dx *= -1  # Move in the opposite direction
        dy *= -1
        new_row = car1_location[0] + dx
        new_col = car1_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with buildings
        if (0 <= new_row < rows and 0 <= new_col < cols) and (new_row, new_col) not in building_locations:
            return (new_row, new_col), (dx, dy)

    # Try each movement until finding a valid one
    for dx, dy in movements:
        new_row = car1_location[0] + dx
        new_col = car1_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with obstacles
        if (0 <= new_row < rows and 0 <= new_col < cols) and \
           ((new_row, new_col) not in building_locations) and \
           ((new_row, new_col) != robot_location) and \
           ((new_row, new_col) != police_location) and \
           ((new_row, new_col) not in car2_image_locations):
            return (new_row, new_col), (dx, dy)

    # If all movements lead to obstacles or out of bounds, return the current location and no movement
    return car1_location, (0, 0)


def move_car2(car2_location, rows, cols, building_locations, robot_location, police_location, car1_image_locations, prev_direction=None):
    """
    Move car2 randomly within the grid, ensuring it does not collide with buildings, robot, police, or other cars.

    Parameters:
        car2_location (tuple): The current location of car2 (row, col).
        rows (int): Number of rows in the grid.
        cols (int): Number of columns in the grid.
        building_locations (list): List of locations for the buildings [(row, col), ...].
        robot_location (tuple): The location of the robot (row, col).
        police_location (tuple): The location of the police (row, col).
        car1_image_locations (list): List of locations for car1 images [(row, col), ...].
        prev_direction (tuple): Previous movement direction of car2 (dx, dy).

    Returns:
        tuple: The new location of car2 (row, col).
        tuple: The new movement direction of car2 (dx, dy).
    """
    # List of possible movements (Right, Left, Down, Up)
    movements = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    # If previous direction is provided and it's in the list of movements, prioritize that direction
    if prev_direction and prev_direction in movements:
        movements.remove(prev_direction)
        movements.insert(0, prev_direction)

    # Randomize the order of movements
    random.shuffle(movements)

    # Check for nearby obstacles
    nearby_entities = []
    for dx, dy in movements:
        new_row = car2_location[0] + dx
        new_col = car2_location[1] + dy

        if (new_row, new_col) == robot_location or \
           (new_row, new_col) == police_location or \
           (new_row, new_col) in car1_image_locations:
            nearby_entities.append((dx, dy))

    # If there are nearby entities, move away from them
    if nearby_entities:
        dx, dy = nearby_entities[0]  # Choose the first nearby entity to move away from
        dx *= -1  # Move in the opposite direction
        dy *= -1
        new_row = car2_location[0] + dx
        new_col = car2_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with buildings
        if (0 <= new_row < rows and 0 <= new_col < cols) and (new_row, new_col) not in building_locations:
            return (new_row, new_col), (dx, dy)

    # Try each movement until finding a valid one
    for dx, dy in movements:
        new_row = car2_location[0] + dx
        new_col = car2_location[1] + dy

        # Check if the new location is within the grid bounds and not colliding with obstacles
        if (0 <= new_row < rows and 0 <= new_col < cols) and \
           ((new_row, new_col) not in building_locations) and \
           ((new_row, new_col) != robot_location) and \
           ((new_row, new_col) != police_location) and \
           ((new_row, new_col) not in car1_image_locations):
            return (new_row, new_col), (dx, dy)

    # If all movements lead to obstacles or out of bounds, return the current location and no movement
    return car2_location, (0, 0)


def main():
    # Initialize Pygame
    pygame.init()

    # Set up the screen
    screen_width, screen_height = 1000, 680
    screen = pygame.display.set_mode((screen_width, screen_height))
    pygame.display.set_caption("Grid with Buildings, Robot, Police, Cars")

    # Load building, robot, police, car1, and car2 images
    building_image = pygame.image.load("building_logo.jpg")
    robot_image = pygame.image.load("robot.jpg")
    police_image = pygame.image.load("police.jpg")
    car1_image = pygame.image.load("car1.jpg")
    car2_image = pygame.image.load("car2.jpg")

    # Define grid size
    rows, cols = 15, 15

    # Generate random building locations
    building_locations = []
    for _ in range(10):
        building_locations.append(generate_location(rows, cols, building_locations))


    # Generate random locations for robot, police, car1, and car2, ensuring they don't overlap with buildings
    robot_location = (0,0)
    police_location = generate_location(rows, cols, building_locations + [robot_location])
    car1_image_locations = []
    for _ in range(2):
        car1_image_locations.append(generate_location(rows, cols, building_locations + [robot_location] + [police_location] + car1_image_locations))
    car2_image_locations = []
    for _ in range(2):
        car2_image_locations.append(generate_location(rows, cols, building_locations + [robot_location] + [police_location] + car1_image_locations + car2_image_locations))

    prev_direction_police = None  # Initialize previous direction for police as None
    prev_direction_car1 = [None, None]  # Initialize previous direction for car1s as None for both cars
    prev_direction_car2 = [None, None]  # Initialize previous direction for car2s as None for both cars

    # Main loop
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Move the police
        new_police_location, prev_direction_police = move_police(police_location, rows, cols, building_locations, robot_location, car1_image_locations, car2_image_locations, prev_direction_police)
        if new_police_location in building_locations or new_police_location == robot_location:
            police_location, prev_direction_police = move_police(police_location, rows, cols, building_locations, robot_location, car1_image_locations, car2_image_locations, prev_direction_police)
        else:
            police_location = new_police_location

        # Move car1 for the first car
        new_car1_location, prev_direction_car1[0] = move_car1(car1_image_locations[0], rows, cols, building_locations, robot_location, police_location, car2_image_locations, prev_direction_car1[0])
        if new_car1_location in building_locations or new_car1_location == robot_location or new_car1_location == police_location:
            car1_image_locations[0], prev_direction_car1[0] = move_car1(car1_image_locations[0], rows, cols, building_locations, robot_location, police_location, car2_image_locations, prev_direction_car1[0])
        else:
            car1_image_locations[0] = new_car1_location

        # Move car1 for the second car
        new_car1_location, prev_direction_car1[1] = move_car1(car1_image_locations[1], rows, cols, building_locations, robot_location, police_location, car2_image_locations, prev_direction_car1[1])
        if new_car1_location in building_locations or new_car1_location == robot_location or new_car1_location == police_location or new_car1_location == car1_image_locations[0]:
            car1_image_locations[1], prev_direction_car1[1] = move_car1(car1_image_locations[1], rows, cols, building_locations, robot_location, police_location, car2_image_locations, prev_direction_car1[1])
        else:
            car1_image_locations[1] = new_car1_location

        # Move car2 for the first car
        new_car2_location, prev_direction_car2[0] = move_car2(car2_image_locations[0], rows, cols, building_locations, robot_location, police_location, car1_image_locations, prev_direction_car2[0])
        if new_car2_location in building_locations or new_car2_location == robot_location or new_car2_location == police_location or new_car2_location == car1_image_locations[0] or new_car2_location == car1_image_locations[1]:
            car2_image_locations[0], prev_direction_car2[0] = move_car2(car2_image_locations[0], rows, cols, building_locations, robot_location, police_location, car1_image_locations, prev_direction_car2[0])
        else:
            car2_image_locations[0] = new_car2_location

        # Move car2 for the second car
        new_car2_location, prev_direction_car2[1] = move_car2(car2_image_locations[1], rows, cols, building_locations, robot_location, police_location, car1_image_locations, prev_direction_car2[1])
        if new_car2_location in building_locations or new_car2_location == robot_location or new_car2_location == police_location or new_car2_location == car1_image_locations[0] or new_car2_location == car1_image_locations[1] or new_car2_location == car2_image_locations[0]:
            car2_image_locations[1], prev_direction_car2[1] = move_car2(car2_image_locations[1], rows, cols, building_locations, robot_location, police_location, car1_image_locations, prev_direction_car2[1])
        else:
            car2_image_locations[1] = new_car2_location

        screen.fill((255, 255, 255))  # Clear the screen
        draw_grid_with_entities(screen, screen_width, screen_height, rows, cols, building_image, building_locations, robot_image, robot_location, police_image, police_location, car1_image, car1_image_locations, car2_image, car2_image_locations)

        pygame.display.flip()  # Update the display
        
        pygame.time.delay(CAR_MOVEMENT_DELAY)  # Introduce delay

    pygame.quit()

if __name__ == "__main__":
    main()
